In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [2]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [3]:
data = pd.read_excel('data_ford_price.xlsx') 

#  Отбор признаков: мотивация

## Предобработка данных

In [4]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [5]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [6]:
x.drop('lat', axis = 1, inplace = True)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [8]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

In [9]:
from sklearn.feature_selection import RFE

In [10]:
y = data['price']
x = data.drop(columns='price')

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [12]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [13]:
X_train.columns

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [14]:
selector.ranking_

array([1, 1, 4, 1, 3, 2])

##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [15]:
from sklearn.feature_selection import SelectKBest, f_regression

In [16]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [17]:
#Выделяем три столбца-признака для обучения, выбранные RFE.
selected_features_rfe = ['year', 'cylinders', 'lat']
X_train_rfe_selected = X_train[selected_features_rfe]
X_test_rfe_selected = X_test[selected_features_rfe]

In [18]:
#Обучаем модель линейной регрессии на выбранных признаках RFE
model_rfe_selected = LinearRegression()
model_rfe_selected.fit(X_train_rfe_selected, y_train)
y_predicted_rfe_selected = model_rfe_selected.predict(X_test_rfe_selected)


In [19]:
#Оцениваем качество модели на выбранных признаках RFE
mae_rfe_selected = mean_absolute_error(y_test, y_predicted_rfe_selected)
print('MAE на выбранных признаках RFE: %.3f' % mae_rfe_selected)

MAE на выбранных признаках RFE: 5096.570


In [20]:
#Выделяем три столбца-признака для обучения, выбранные SelectKBest
selected_features_kbest = ['year', 'cylinders', 'odometer']
X_train_kbest_selected = X_train[selected_features_kbest]
X_test_kbest_selected = X_test[selected_features_kbest]

In [21]:
#Обучаем модель линейной регрессии на выбранных признаках SelectKBest
model_kbest_selected = LinearRegression()
model_kbest_selected.fit(X_train_kbest_selected, y_train)
y_predicted_kbest_selected = model_kbest_selected.predict(X_test_kbest_selected)

In [22]:
#Оцениваем качество модели на выбранных признаках SelectKBest
mae_kbest_selected = mean_absolute_error(y_test, y_predicted_kbest_selected)
print('MAE на выбранных признаках SelectKBest: %.3f' % mae_kbest_selected)

MAE на выбранных признаках SelectKBest: 4708.946


In [23]:
#Сравниваем выбранные метрики
if mae_rfe_selected < mae_kbest_selected:
    print('Метод RFE показал лучший результат на тестовой выборке.')
elif mae_rfe_selected > mae_kbest_selected:
    print('Метод SelectKBest показал лучший результат на тестовой выборке.')
else:
    print('Оба метода показали одинаковое качество на тестовой выборке.')

Метод SelectKBest показал лучший результат на тестовой выборке.


***
### Вывод:

#### Сравнив результаты оценки качества моделей линейной регрессии с использованием методов RFE и SelectKBest на тестовой выборке, мы получили следующие результаты:


|  MAE на выбранных признаках SelectKBest     | 4708.946     |
|---------------------------------------------|:------------:|
|  MAE на выбранных признаках RFE             |  5096.570    |

#### Метод SelectKBest показал лучший результат на тестовой выборке, имея меньшее значение MAE. Это означает, что модель, построенная на признаках, выбранных с использованием метода SelectKBest, лучше предсказывает целевую переменную (цену) на тестовых данных по сравнению с моделью, построенной на признаках, выбранных методом RFE.

***